### Data Pre-Processing


1. **Obtaining the data -** data is loaded from local machine


2. **Cleaning the data -** preforming some text pre-processing techniques


3. **Organizing the data -** organizing the cleaned data into a way that is easy to input into other algorithms

In [1]:
#Importing packages
import os
import glob
import nltk
import pandas as pd
import numpy as np

### 1- Obtaining the data

#### Labling Data For All Organizations

In [2]:
# 1- For Guilford-County data 
GC_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\GuilfordCounty_original_data.csv', engine='python')
GC_df = GC_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
GC_df.insert(2, "organization", "Guilford County")


# 2- For Charlotte-City data
CC_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\CharlotteCity_original_data.csv', engine='python')
CC_df = CC_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
CC_df.insert(2, "organization", "Charlotte City")

# 3- For Durham-City data
DCity_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\DurhamCity_original_data.csv', engine='python')
DCity_df = DCity_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
DCity_df.insert(2, "organization", "Durham City")

# 4- For Durham-County data
DCounty_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\DurhamCounty_original_data.csv', engine='python')
DCounty_df = DCounty_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
DCounty_df.insert(2, "organization", "Durham County")

# 5- For Mecklenburg-County data
MC_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\MecklenburgCounty_original_data.csv', engine='python')
MC_df = MC_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
MC_df.insert(2, "organization", "Mecklenburg County")

# 6- For Raleigh-City data
RC_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\RaleighCity_original_data.csv', engine='python')
RC_df = RC_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
RC_df.insert(2, "organization", "Raleigh City")

# 7- For Wake-County data
WC_df = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\original\WakeCounty_original_data.csv', engine='python')
WC_df = WC_df.drop(columns="Unnamed: 0")
# inserting column with static value in data frame 
WC_df.insert(2, "organization", "Wake County")

#### Converting Data Frames into CSV Files

In [3]:
GC_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\GuilfordCounty.csv')
CC_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\CharlotteCity.csv')
DCity_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\DurhamCity.csv')
DCounty_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\DurhamCounty.csv')
MC_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\MecklenburgCounty.csv')
RC_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\RaleighCity.csv')
WC_df.to_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\WakeCounty.csv')

#### Setting the Working Directory to Combine Data Files

In [4]:
os.chdir(r"C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed")

#### Using glob to Match the Pattern ‘csv’

In [5]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#### Combining All Data Files

In [6]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "CombinedData2019-2020.csv", index=False, encoding='utf-8-sig')

In [7]:
#Change the dirctory back
os.chdir(r"C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\src")

#### Reading Data

In [8]:
data = pd.read_csv(r'C:\Users\Sultan\Documents\GitHub\Budget_Text_Analysis\util\data\2019-2020\structured\processed\CombinedData2019-2020.csv', engine='python')

### 2- Cleaning the Data

In [9]:
# listing columns in data frame 
list(data)

['ï»¿Unnamed: 0', 'page_number', 'word', 'organization']

#### Dropping a Column Added Upon Combining the Data Files 

In [10]:
# alternatively, delete columns using the columns parameter of drop
data = data.drop(columns="ï»¿Unnamed: 0")
data.head()

,page_number,word,organization
0,1,ensuring,Charlotte City
1,1,an,Charlotte City
2,1,equitable,Charlotte City
3,1,sustainable,Charlotte City
4,1,and,Charlotte City


#### Adding Year Colunm With A Static Value = 2019-2020

In [11]:
data.insert(3, "year", "2019-2020")
data.head()

,page_number,word,organization,year
0,1,ensuring,Charlotte City,2019-2020
1,1,an,Charlotte City,2019-2020
2,1,equitable,Charlotte City,2019-2020
3,1,sustainable,Charlotte City,2019-2020
4,1,and,Charlotte City,2019-2020


####  Lower casing the words

In [19]:
data['word'] = data['word'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['organization'] = data['organization'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data.head()

,page_number,word,organization,year
0,1,ensuring,charlotte city,2019-2020
1,1,,charlotte city,2019-2020
2,1,equitable,charlotte city,2019-2020
3,1,sustainable,charlotte city,2019-2020
4,1,,charlotte city,2019-2020


#### Lemmatization

In [13]:
from textblob import Word
data['word'] = data['word'].apply(lambda x: " ".join([Word(x).lemmatize() for x in x.split()]))

data.head()

,page_number,word,organization,year
0,1,ensuring,Charlotte City,2019-2020
1,1,an,Charlotte City,2019-2020
2,1,equitable,Charlotte City,2019-2020
3,1,sustainable,Charlotte City,2019-2020
4,1,and,Charlotte City,2019-2020


#### Removing stop words

In [17]:
# Load library
from nltk.corpus import stopwords

# Download the set of stop words the first time
nltk.download('stopwords')

# Load stop words
stop_words = stopwords.words('english')

data['word'] = data['word'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))
data['word'].head()

0       ensuring
1               
2      equitable
3    sustainable
4               
Name: word, dtype: object

### 2- Organizing the Data

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [20]:
# Let's take a look at our dataframe
data

,page_number,word,organization,year
0,1,ensuring,charlotte city,2019-2020
1,1,,charlotte city,2019-2020
2,1,equitable,charlotte city,2019-2020
3,1,sustainable,charlotte city,2019-2020
4,1,,charlotte city,2019-2020
...,...,...,...,...
2552519,498,index,wake county,2019-2020
2552520,498,fiscal,wake county,2019-2020
2552521,498,year,wake county,2019-2020
2552522,498,adopted,wake county,2019-2020


In [21]:
# Pickle it for later use
data.to_pickle("corpus.pkl")